# cost function

For every community, the cost function is defined as $weight \times distance^2$. Namely, the cost function is $w ((x - x_0)^2 + (y - y_0)^2)$

In [2]:
import numpy as np

In [3]:
def single_cost_func(x_old: float, y_old: float) -> np.ndarray:
    '''
        x and y are the position of one single clinic in the coordinate, 
        The return value stores the coefficients of [x^2, x, y^2, y, 1]
    '''
    return np.array([1, -2 * x_old, 1, -2 * y_old, x_old^2 + y_old^2])

def cost_func(w_list: list, x_list: list, y_list: list):
    '''
        w_list stores the list of weight, 
        x_list stores the list of the x-axis, 
        y_list stores the list of the y-axis, 
        cost_func_coeff is the total sum of coefficients
    '''
    try:
        cost_func_coeff = np.array(np.zeros((1, 5))) # 5 means the numbers of coefficients
        # optimization
        for i in range(len(w_list)):
            cost_func_coeff += w_list[i] * single_cost_func(x_list[i], y_list[i]) # Addition between numpy arrays
        return cost_func_coeff
    except Exception as e:
        print('Sum of w_list: {}'.format(np.sum(np.array(w_list))))
        print('ERROR in cost_func: w_list = {}, x_list = {}, y_list = {}'.format(len(w_list), len(x_list), len(y_list))) 
        print(e)
    

cost_func([1,1,1], [1,2,3], [3,2,1])

array([[  3., -12.,   3., -12.,  12.]])